In [1]:
# import sys
# !{sys.executable} -m pip install tavily-python
# !{sys.executable} -m pip install GoogleNews
# !pip install GoogleNews
# !pip install gnews

In [2]:
from dotenv import load_dotenv
import os
from tavily import TavilyClient
from langchain.chat_models import init_chat_model
import requests
from GoogleNews import GoogleNews
from datetime import datetime
import import_ipynb
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

In [3]:
from get_stock_data import get_historical_prices_data,get_current_prices

In [4]:
# Load environment variables from .env
load_dotenv()
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

In [5]:
def web_search(query : str):
    '''
    This Function searches the web and return its responce
    '''
    
    client = TavilyClient(api_key=TAVILY_API_KEY)
    responce = client.search(query=query, search_depth="advanced")
    return responce

In [6]:
def get_recent_news(comp_ticker: str, total_news: int = 10):
    googlenews = GoogleNews(lang='en')
    
    # Set a time range — last 3 days
    end = datetime.now()
    start = end - timedelta(days=1)
    googlenews.set_time_range(start.strftime('%m/%d/%Y'), end.strftime('%m/%d/%Y'))

    googlenews.search(f'{comp_ticker} Stock News')
    
    results = []
    titles_seen = set()
    page = 1

    while len(results) < total_news:
        googlenews.get_page(page)
        page_results = googlenews.results()[:10]  # last 10 results on the page

        for item in page_results:
            if item['title'] not in titles_seen:
                results.append(item)
                titles_seen.add(item['title'])
            if len(results) == total_news:
                break
        page += 1
        if page > 5:  # safety break
            break

    return results

In [7]:
llm = init_chat_model('google_genai:gemini-2.0-flash')

def get_total_analysis(comp_ticker : str, period : str, interval : str):


    results = get_recent_news(comp_ticker, total_news=10)
    final_response = []
    for result in results:
        
        # print(f"**Title** : {result['title']}\n\t-Published : {(result['date'])}\n\t-Description : {result['desc']:>20}\n")
        final_response.append(f"**Title** : {result['title']}\n\t-Published : {(result['date'])}\n\t-Description : {result['desc']:>20}\n")
    
    formatted_news = "\n\n".join(final_response)

    
    price_data = get_historical_prices_data(comp_ticker, period, interval)
    formatted_price_data = price_data.to_string()

    
    prompt = f"""
    📊 You are a **Financial Analyst AI**. Based on the following raw stock data and recent news about **{comp_ticker}**, generate a detailed and well-structured analysis.
    
    ---
    
    ## 📂 Raw Stock Price Dataset (Date, Open, High, Low, Close, Volume)
    {formatted_price_data}
    
    ---
    
    ## 📰 Recent News Headlines
    {formatted_news}
    
    ---
    
    ## 📊 Stock Performance Summary
    
    Using the raw stock data above, calculate and summarize the following:
    
    - **Trend**: Is the stock price generally increasing, decreasing, or flat? Summarize the trend direction over the period.
    - **Volatility**: Describe the level of price fluctuation (high/medium/low) based on daily swings.
    - **Returns**: Calculate total and average return over the period. Express in percentage.
    - **Volume Insights**: Describe how trading volume has changed (rising, falling, or consistent), and what that might indicate.
    
    ---
    
    ## 📈 Technical Indicator Snapshot
    
    From the dataset above, compute:
    
    - **RSI (14-day)**: Provide RSI value and interpretation (Overbought/Oversold/Neutral)
    - **MACD**: Is the MACD line above/below the signal line? Indicate whether it's a bullish or bearish signal.
    - **SMA (50-day)**: Compare the latest price with the 50-day Simple Moving Average. Indicate if price is above/below SMA.
    - You may include any **other relevant indicators** or patterns found (e.g., Bollinger Bands, Moving Averages crossover).
    
    ---
    
    Now, based on both technical data and news:
    
    ### 1.) Overall News Sentiment
    - One-word sentiment: **Positive**, **Neutral**, or **Negative**
    - Describe the emotional tone of the news coverage.
    
    ---
    
    ### 2.) Key News Drivers
    List major news themes (if any), such as:
    - 🏭 Product Launches or Innovations
    - 📊 Financial Earnings or Guidance
    - ⚖️ Regulatory or Legal Issues
    - 🌍 Market Expansion or Partnerships
    - 👥 Leadership or Organizational Changes
    
    ---
    
    ### 3.) Short-Term Outlook (1–4 Weeks)
    - Expected direction: 📈 Rise / 📉 Fall / ⚠️ Remain Volatile
    - Brief explanation based on technicals and news.
    
    ---
    
    ### 4.) Long-Term Outlook (6+ Months)
    Evaluate:
    - Growth potential
    - Innovation pipeline
    - Industry trends
    - Strategic risks
    
    ---
    
    ### 5.) Final Investment Recommendation
    Make a final call: **Buy / Hold / Sell**
    - Use both technical + news reasoning.
    
    ---
    
    🎯 Use markdown formatting (headers, bold, bullet points). Provide numerical calculations where relevant. Be structured and concise.
    """

    response = llm.invoke(prompt)
    return response.content
